In [458]:
import sys
sys.path.append('/Users/huibmeulenbelt/PycharmProjects/ufc')
sys.path.append('/Users/huibmeulenbelt/PycharmProjects/ufc/scripts')
sys.path.append('/Users/huibmeulenbelt/PycharmProjects/ufc/scripts/markov')

In [459]:
import json
import numpy as np
from networkx.readwrite import json_graph
from fight import Fight

In [460]:
PATH_G = '/Users/huibmeulenbelt/PycharmProjects/ufc/scripts/markov/tests/12120.json'
G = json_graph.adjacency_graph(json.load(open(PATH_G)))

In [461]:
n_rounds = 3
fight = Fight(G=G, n_rounds=n_rounds)

The goal is to calculate the probability of visiting transient state(s) $j$ at least once before being absorbed in state(s) $k$ in $n$ steps.

$$
P(T_j < T_k) \cdot P(\text{absorbed in k} | T_j < T_k)
$$


In [462]:
j = 4
j += fight.i_choke_red

In [463]:
k = [fight.i_submission_red]

In [464]:
mu = np.zeros_like(fight.mu)
mu[:, j] = 1
mu

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], shape=(4000, 39))

In [465]:
p_ij = fight.h(j=j).squeeze(1)

In [466]:
p_jk = fight.h(j=k, mu=mu)
p_jk = np.sum(p_jk, axis=1)

In [467]:
mask = np.triu(np.ones((fight.n, fight.n)))[::-1, :]

In [468]:
cum_p_jk = np.einsum('st,it->si', p_jk, mask)

In [474]:
np.sum(cum_p_jk, axis=1)

array([1.29627081, 0.58115743, 0.82162398, ..., 1.42429823, 1.01470326,
       3.81579064], shape=(4000,))

In [469]:
p_comeback_blue = np.sum(p_ij * cum_p_jk, axis=1)
p_comeback_blue

array([2.78352751e-05, 7.84517635e-06, 9.25308843e-06, ...,
       7.52218124e-05, 9.10501883e-06, 4.23986162e-04], shape=(4000,))

In [470]:
p_submission_red = fight.p_submission_red
p = np.sum(p_submission_red, axis=1)
p

array([0.00336983, 0.00105849, 0.00153929, ..., 0.00637246, 0.00160636,
       0.01231867], shape=(4000,))

In [472]:
bool(np.all(p_comeback_blue < p_won_blue))

True